In [ ]:
#| default_exp react

In [1]:
#| export
import re

from fastcore.all import *

from chat_react.core import *

In [132]:
#| export
class CancelCallTool(Exception): pass
class CancelStep(Exception): pass
class CancelLLM(Exception): pass
class CancelParse(Exception): pass

In [170]:
#| export
class LogMessagesCB:
    def after_llm(self, bot, out): print(out)
    def after_call_tool(self, bot, out): print(f'Output: {out}')

In [171]:
#| export
class React:
    def __init__(self, prompt, examples, tools=[], cbs=[], max_turns=10, end_tool='finish'):
        self.prompt = prompt
        # self.tools = tools
        store_attr()
        self.messages = [hm(prompt)] + examples + [aim("Though: I have to greet the user\nAction: send_to_user[Hi, I am an AI, how can I help you today?]")]
    
    def llm(self, messages): return chat_create(messages)

    def with_cbs(self, name, exception, *args, **kwargs):
        for cb in self.cbs:
            if hasattr(cb, f'before_{name}'): getattr(cb, f'before_{name}')(self, *args, **kwargs)
        try: result = getattr(self, name)(*args, **kwargs) 
        except exception as e:
            for cb in self.cbs:
                if hasattr(cb, f'after_cancel_{name}'): getattr(cb, f'after_cancel_{name}')(self, *args, **kwargs)
            return
        for cb in self.cbs:
            if hasattr(cb, f'after_{name}'): getattr(cb, f'after_{name}')(self, result)
        return result
    
    def __call__(self, query):
        return self.run(self.messages + [hm(f'Output: {query}')])
    
    def run(self, messages):
        for i in range(self.max_turns):
            messages, tool_name, tool_input = self.with_cbs('step', CancelStep, messages)
            if tool_name == self.end_tool: break
        return messages, tool_name, tool_input


    def step(self, messages):
        out = self.with_cbs('llm', CancelLLM, messages)
        messages.append(aim(out))
        tool_name, tool_input = self.with_cbs('parse_output', CancelParse, out)
        if tool_name != self.end_tool:
            tool_output = self.with_cbs('call_tool', CancelCallTool, tool_name, tool_input)
            messages.append(hm(f"Output: {tool_output}"))
        return messages, tool_name, tool_input
    
    def parse_output(self, text):
        """Parses text into action and action_input."""
        i = text.find('Action:')
        m = re.match(r"Action: (?P<tool_name>.*)\[(?P<tool_input>.*)\]", text[i:], re.DOTALL)
        if not m: raise Exception
        return m.group(1, 2)

    def call_tool(self, tool_name, tool_input):
        for tool in self.tools: 
            if tool.__name__ == tool_name: return tool(tool_input)
    
    @classmethod
    def from_funs(cls, funs, cbs=[]):
        tools = [{fun.__name__: fun} for fun in funs]
        if 'send_to_user' not in tools: tools.append({'send_to_user': input})
        if 'finish' not in tools: tools.append({'finish': None})
        prompt = "You can use the following tools:\n" + '\n'.join([f"- {list(tool.keys())[0]}" for tool in tools])
        prompt += "Before using a tool think about what you want to do and write it down in a sentence.\n"
        prompt += "Use the following format:\nThougth: <your thought>\nAction: <tool_name>[<tool_input>]\n\n"
        examples = [aim("Though: I have to great the user\nAction: send_to_user[Hi, I am an AI, how can I help you today?]"),
                    hm("Output: how much is 45+32"),
                    aim("Though: I have to calculate 45+32\nAction: calculator[45+32]"),
                    hm("Output: 77"),aim("Though: I know the answer to the user question\nAction: finish[77]")]
        return cls(prompt=prompt, examples=examples, tools=funs, cbs=cbs)

In [172]:
#| export
def calculator(s):
    """python interpreter"""
    return eval(s)

In [173]:
bot = React.from_funs([calculator], cbs=[LogMessagesCB()])

In [186]:
x = 1951
y = 95109
_, _, out = bot(f"how much is {x}*{y}")
print(out)
assert int(out) == x*y

Though: I have to calculate 1951*95109
Action: calculator[1951*95109]
Output: 185557659
Though: I know the answer to the user question
Action: finish[185557659]
185557659


In [184]:
out = chat_create([hm(f"how much is {x}*{y}. output only the result with no commas")])
assert int(out)==x*y

AssertionError: 

In [187]:
#| hide
import nbdev; nbdev.nbdev_export()